In [1]:

from transformers import AutoConfig
from safetensors.torch import load_file
import os, gc, json, logging
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from safetensors.torch import load_file

logging.getLogger("transformers").setLevel(logging.ERROR)

from tsfm_public.models.tspulse.modeling_tspulse import TSPulseForReconstruction, get_fft


In [2]:
data_path = "../Dataset/AnomalyDetection/train.csv"


## Helper Functions for Data Loading and Processing

`load_model` loads the model when checkpoints is passed else load the base EnergyTSPulse

In [ ]:
def load_model(ckpt_path=None, device="cpu"):
    """
    Loads TSPulse model and prepares it for inference.
    (Prefix cleaning code kept commented as in original script)
    """
    if ckpt_path is not None:
        print(f"[INFO] Loading TSPulse model from checkpoint: {ckpt_path}")
        config = AutoConfig.from_pretrained(ckpt_path)
        model = TSPulseForReconstruction.from_pretrained(
            ckpt_path,
            config=config,
        )
    else:
        print("[INFO] Loading pretrained TSPulse model.")
        model = TSPulseForReconstruction.from_pretrained(
            "../Energy-TSPulse/Checkpoint"
        )
    if isinstance(device, str):
        device = torch.device(device)

    print("[INFO] Initialized TSPulse pretrained model.")
    model.to(device)
    model.eval()

    P = model.config.context_length
    stride = model.config.patch_stride

    print(f"[INFO] Model Ready | Context Length (P) = {P} | Patch Stride = {stride}")
    
    return model, P, stride







In [4]:
csv_path = "../Dataset/AnomalyDetection/train.csv"   # change path here directly
ckpt_dir = None # checkpoint folder (not used now but left for API consistency)
gpu_id   = 0

device = f"cuda:{gpu_id}" if torch.cuda.is_available() else "cpu"
print(f"[INFO] Device selected: {device}")

df = pd.read_csv(csv_path, parse_dates=["timestamp"])
df.sort_values(["building_id", "timestamp"], inplace=True)

# Fill missing readings per building (using median fallback)
df["meter_reading"] = df.groupby("building_id")["meter_reading"] \
                        .transform(lambda s: s.fillna(s.median()))
print("[INFO] Filled missing meter_reading per building using median.")


[INFO] Device selected: cuda:0
[INFO] Filled missing meter_reading per building using median.


In [5]:
df

,building_id,timestamp,meter_reading,anomaly
0,1,2016-01-01 00:00:00,38.651,0
200,1,2016-01-01 01:00:00,38.651,0
398,1,2016-01-01 02:00:00,38.651,0
597,1,2016-01-01 03:00:00,38.651,0
796,1,2016-01-01 04:00:00,38.651,0
...,...,...,...,...
1748693,1353,2016-12-31 19:00:00,2.425,0
1748893,1353,2016-12-31 20:00:00,2.450,0
1749093,1353,2016-12-31 21:00:00,2.425,0
1749293,1353,2016-12-31 22:00:00,2.450,0


## Loading ZeroShort TSPulseReconstruction Model

In [6]:
# path_to_tspulse_model = "../Energy-TSPulse/Pretraining/Checkpoint"
# zeroshot_model = TSPulseForReconstruction.from_pretrained(
#     path_to_tspulse_model,
#     num_input_channels=1,
#     revision="main",
#     mask_type="user",
# )
model, P, stride = load_model(ckpt_dir, device)

[INFO] Loading pretrained TSPulse model.
[INFO] Initialized TSPulse pretrained model.
[INFO] Model Ready | Context Length (P) = 512 | Patch Stride = 8


## Computing Anomaly Score With Anomaly Pipeline 

In [7]:
def calc_window_metrics(y_true, y_pred, window_size):
    """
    Computes window-wise anomaly classification metrics.
    A window is anomalous if it contains ANY anomaly.
    """
    n_blocks = len(y_true) // window_size
    TP, FP, FN = 0, 0, 0
    
    for i in range(n_blocks):
        s, e = i * window_size, (i + 1) * window_size
        true_any = y_true[s:e].any()
        pred_any = y_pred[s:e].any()

        if true_any and pred_any:
            TP += 1
        elif not true_any and pred_any:
            FP += 1
        elif true_any and not pred_any:
            FN += 1

    prec = TP/(TP+FP) if (TP+FP)>0 else 0
    rec  = TP/(TP+FN) if (TP+FN)>0 else 0
    f1   = 2*prec*rec/(prec+rec) if (prec+rec)>0 else 0

    return prec, rec, f1


In [8]:
def find_best_threshold(scores, y_true, window_size):
    """
    Searches threshold to maximize window F1.
    Only scans over scores from normal windows to ignore extreme outliers.
    """
    best_f1, best_th = -1, 0

    y_win = y_true.reshape(-1, window_size)
    normal_scores = scores[~y_win.any(axis=1)]

    if len(normal_scores) < 2:
        lo, hi = scores.min(), scores.max()
    else:
        lo, hi = normal_scores.min(), normal_scores.max()

    if hi <= lo:
        hi = lo + 1e-6

    for th in np.linspace(lo, hi, 100):
        block_pred = (scores > th).astype(int)
        y_pred = np.repeat(block_pred, window_size)
        _, _, f1 = calc_window_metrics(y_win.reshape(-1), y_pred[:len(y_true)], window_size)
        if f1 > best_f1:
            best_f1, best_th = f1, th

    return best_th


In [9]:
def evaluate_model(model, df, context_length, device, fft_weight=0.5):
    model.eval()
    out_rows = []

    for bid in df.building_id.unique():
        sub = df[df.building_id == bid]
        x = sub.meter_reading.to_numpy(np.float32)
        y = sub.anomaly.to_numpy(int)
        n = len(x)

        if n < context_length or x.std() < 1e-5:
            continue

        mu, sd = x.mean(), x.std()
        xn = (x-mu)/sd
        pad = (-n) % context_length
        if pad:
            xn = np.concatenate([xn, np.zeros(pad, np.float32)])
            y  = np.concatenate([y,  np.zeros(pad, int)])

        scores = []
        for i in range(0, len(xn), context_length):
            w = xn[i:i+context_length]
            t = torch.tensor(w, device=device).view(1, context_length, 1)
            m = torch.ones_like(t, dtype=torch.bool)

            with torch.no_grad():
                out = model(past_values=t, past_observed_mask=m)

            rec = out.reconstruction_outputs.view(-1).cpu().numpy()
            se_time = ((rec-w)**2).max()

            fft_true, *_ = get_fft(t)
            fft_rec = out.reconstruction_outputs  # using time head FFT recon
            se_freq = ((fft_rec-fft_true)**2).max().item()

            scores.append(se_time + fft_weight*se_freq)

        sc = np.array(scores, np.float32)
        th = find_best_threshold(sc, y[:len(sc)*context_length], context_length)
        y_pred = np.repeat((sc>th).astype(int), context_length)[:n]

        wp, wr, wf = calc_window_metrics(y[:n], y_pred, context_length)
        out_rows.append({"building_id":bid, "win_precision":wp, "win_recall":wr, "win_f1":wf})

    return pd.DataFrame(out_rows)


## Zero Shot

In [10]:


print("[INFO] Running zero-shot anomaly detection evaluation for all buildings...\n")

eval_results = evaluate_model(model, df, P, device,)

if not eval_results.empty:
    print("\n==================== ZERO-SHOT RESULTS ====================")
    print(f"Average Window F1: {eval_results.win_f1.mean():.4f} ± {eval_results.win_f1.std():.4f}")
    print(f"Average Window Precision: {eval_results.win_precision.mean():.4f} ± {eval_results.win_precision.std():.4f}")
    print(f"Average Window Recall: {eval_results.win_recall.mean():.4f} ± {eval_results.win_recall.std():.4f}")

    # Save results
    # save_csv = "tspulse_zeroshot_eval_results.csv"
    # eval_results.to_csv(save_csv, index=False)
    # print(f"\n[INFO] Saved results to: {save_csv}")

else:
    print("[WARN] No valid buildings passed filtering for evaluation.")


[INFO] Running zero-shot anomaly detection evaluation for all buildings...


==================== ZERO-SHOT RESULTS ====================
Average Window F1: 0.7998 ± 0.1547
Average Window Precision: 0.7611 ± 0.2054
Average Window Recall: 0.9030 ± 0.1474


In [11]:
class AnomalyDataset(Dataset):
    """
    Creates per-building fixed windows without modifying timestamps.
    Used for training (self-supervised) and evaluation.
    """
    def __init__(self, df, context_length, training=True):
        self.context_length = context_length
        self.training = training
        self.samples = []
        self.process_data(df)

    def process_data(self, df):
        for bid, sub in df.groupby("building_id"):
            x = sub["meter_reading"].to_numpy(np.float32)
            y = sub["anomaly"].to_numpy(int)

            if len(x) < self.context_length or x.std() < 1e-5:
                continue

            mu, sd = x.mean(), x.std()
            xn = (x - mu)/sd

            for i in range(0, len(xn)-self.context_length+1, self.context_length):
                self.samples.append({
                    "past_values": torch.tensor(xn[i:i+self.context_length]).unsqueeze(-1),
                    "anomaly_labels": torch.tensor(y[i:i+self.context_length], dtype=torch.bool)
                })

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        s = self.samples[idx]
        if self.training:
            mask = torch.ones(self.context_length, dtype=torch.bool)
            loss_mask = ~s["anomaly_labels"]
            return {
                "past_values": s["past_values"],
                "past_observed_mask": mask.unsqueeze(-1),
                "loss_mask": loss_mask.unsqueeze(-1)
            }
        return {"past_values": s["past_values"], "anomaly_labels": s["anomaly_labels"]}
